## Финальное соревнование

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

В данном задании вас ждет неизвестная зависимость. Ваша основная задача: **построить две лучших модели**, минимизирующих среднеквадратичную ошибку (MSE):
1. На первую модель не налагается ограничений.
2. Вторая модель должна быть **линейной**, т.е. представлять собой линейную комбинацию признаков плюс свободный член: $\boldsymbol{w}^{\top}\boldsymbol{x} + b$. При этом __вы можете использовать базовые математические операции для преобразования признаков__: np.exp, np.log, np.pow (полный список доступен в [документации](https://numpy.org/doc/stable/reference/routines.math.html)), а также линейные операции над ними (сумма, умножение на число и пр.). Для преобразования признаков вам будет необходимо написать функцию `my_transformation`. __Кол-во параметров (весов) используемых второй моделью не должно превышать 15 (включая свободный член).__

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на готовые примеры, а не просто "скопировать-вставить". Это поможет вам в будущем писать код более уверенно

In [1]:
import os
import json

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from pandas.plotting import scatter_matrix
import seaborn as sns


from scipy import stats
from scipy.stats import norm

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler

Загрузка данных происходит ниже. Если она не срабатывает, самостоятельно скачайте файл `hw_final_open_data.npy` и положите его в ту же директорию, что и ноутбук.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_target.npy

In [ ]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

Разбивка на `closed_data` и `val` опциональна и сделана для вашего удобства.

In [ ]:
closed_data_x, valid_x, closed_data_y, valid_y = train_test_split(data, target, test_size=0.3)

### Модель №1
Напоминаем, в первой части задания ваша основная задача – получить наилучший результат без ограничений на модель. Сдаваться будут только предсказания модели.

Пример с использованием Random Forest доступен ниже.

In [ ]:
rf = RandomForestRegressor()
rf.fit(closed_data_x, closed_data_y)

print(
    f'closed_data mse =\t {mean_squared_error(np.round(rf.predict(closed_data_x), 2), np.round(closed_data_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(rf.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

##### Сдача первой части соревнования
Загрузите файл `hw_final_closed_data.npy` (ссылка есть на странице с заданием). Если вы используете sklearn-совместимую модель, для генерации посылки вы можете воспользоваться функцией `get_predictions`. В ином случае перепишите функцию для вашей модели и запустите код под следующей ячейкой для генерации посылки.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy -O hw_final_closed_data.npy

In [ ]:
assert os.path.exists('hw_final_closed_data.npy'), 'Please, download `hw_final_closed_data.npy` and place it in the working directory'
closed_data = np.load('hw_final_closed_data.npy', allow_pickle=False)

Если необходимо, преобразуйте данные. Преобразованную матрицу объект-признак сохраните в переменную `closed_data`.

In [ ]:
data.shape

In [ ]:
closed_data.shape

In [ ]:
data_df = pd.DataFrame(data)
data_df.rename(columns={0: '0', 1: '1', 2: '2' , 3: '3', 4: '4', 5: '5', 6: '6'}, inplace=True)
data_df.head()

In [ ]:
closed_data = pd.DataFrame(closed_data)
closed_data.rename(columns={0: '0', 1: '1', 2: '2' , 3: '3', 4: '4', 5: '5', 6: '6'}, inplace=True)
closed_data.head()

In [ ]:
def get_outliers(df: pd.core.frame.DataFrame, column: str) -> tuple:
    q_one = df[column].quantile(q=.25)
    q_three  = df[column].quantile(q=.75)
    iqr = q_three - q_one
    left_border = q_one-(1.5*iqr)
    right_border = q_three+(1.5*iqr)
    outliers_left = df[df[column] < left_border]
    outliers_right = df[df[column] > right_border]
    full_outliers = len(outliers_left) + len(outliers_right)
    return(full_outliers, left_border, right_border)

In [ ]:
def plot_charts(df_columns: list) -> None:
    num_chrt = len(df_columns)
    for column in df_columns:

        outliers_full, left_border_full, right_border_full = get_outliers(data_df, column)
        fig = plt.figure(figsize=[20, 15])
        grd = plt.GridSpec(ncols=1,nrows=2, top=0.92, wspace=0.15, hspace=0)
        plt.suptitle(f'Boxplot and histplot for "{str(column).upper()}"', fontsize=17)
        
        fig_ax_1 = fig.add_subplot(grd[1,0])
        plt.grid(True)
        sns.set_style('darkgrid')
        hist = sns.histplot(data=data_df, x=column,
                            multiple='stack',
                            alpha=0.25, legend=False, kde=True,
                            linewidth=3)
        for line in hist.lines:
            line.set_linewidth(5)
        median_line_full_data_df = plt.axvline(np.median(data_df[column]), 
                                             color='indigo', 
                                             linestyle='-',
                                             lw=2,   
                                             label=f'Median line of feature')
        left_border_line_full_data_df= plt.axvline(left_border_full, 
                                                  color='m', 
                                                  linestyle=':',
                                                  lw=3,
                                                  label=f'First quantile of {str(column).upper()} for full feature')
        right_border_line_full_data_df = plt.axvline(right_border_full, 
                                                   color='m', 
                                                   linestyle='-.',
                                                   lw=3,
                                                   label=f'Third quantile of {str(column).upper()} for full feature') 
        plt.xlabel('Values', fontsize=15)
        plt.ylabel('Counts', fontsize=15)

        
        fig_ax_2 = fig.add_subplot(grd[0,0])
        sns.set_style('darkgrid')
        plt.grid(True)
        ax = sns.boxplot(x=column, data=data_df, palette=['blue'],
                         orient="h", showcaps=False, notch=True, medianprops={"color": "indigo"}, boxprops=dict(alpha=.5))
        mean_line_full_data_df = plt.axvline(np.mean(data_df[column]), 
                                           color='black', 
                                           linestyle='--',
                                           lw=3,
                                           label=f'Mean line of feature')
        median_line_full_data_df= plt.axvline(np.median(data_df[column]), 
                                             color='indigo', 
                                             linestyle='-',
                                             lw=3,   
                                             label=f'Median line of feature')
        left_border_line_full_data_df = plt.axvline(left_border_full, 
                                                  color='m', 
                                                  linestyle=':',
                                                  lw=3,
                                                  label=f'First quantile of {str(column).upper()} for full feature')
        right_border_line_full_data_df = plt.axvline(right_border_full, 
                                                   color='m', 
                                                   linestyle='-.',
                                                   lw=3,
                                                   label=f'Third quantile of {str(column).upper()} for full feature') 
        plt.ylabel(f'{column}', fontsize=15)
        plt.gca().axes.get_xaxis().set_visible(False)
        
        plt.legend(title=f'Legend for "{str(column).upper()}"',
                   handles=[Patch(color='blue', alpha=.5, label='Full feature'),
                            Patch(color='red', alpha=.5, label='Disaster'),
                            Patch(color='green', alpha=.5, label='Not disaster'),
                            mean_line_full_data_df,
                            median_line_full_data_df,
                            left_border_line_full_data_df,
                            right_border_line_full_data_df,
                            Line2D([0], [0], color='green', lw=2, label='Distibution for disaster for not disaster'),
                            Line2D([0], [0], color='red', lw=2, label='Distibution for disaster'),
                            Patch(color='none', label=f'Num outliers full-{outliers_full:.2f}'),
                            Patch(color='none', label=f'Std value-{data_df[column].std():.2f}'),
                            Patch(color='none', label=f'Mean value-{np.mean(data_df[column]):.2f}'),
                            Patch(color='none', label=f'Median value-{np.median(data_df[column]):.2f}'),],
                   
                   edgecolor = 'r',
                   facecolor = 'oldlace',
                   ncol=3,
                   title_fontsize=17,
                   fontsize=15,
                   loc='center',
                   bbox_to_anchor=(0.5, -1.45));

In [ ]:
def plot_dist(feature: str) -> None:
    plt.figure(figsize=[15, 7])
    plt.subplots_adjust(top=0.88, wspace=0.3, hspace=0.95)
    plt.suptitle('Distibution and Probalities', fontsize=15)
    sns.set_style('darkgrid')
    
    plt.subplot(2, 2, 1)
    plt.title('Distibution for train', fontsize=13)
    mu, std = np.mean(data_df[feature]), np.std(data_df[feature])
    x = np.linspace(mu - 3*std, mu + 3*std, 100)
    pdf = norm.pdf(x, mu, std)
    sns.kdeplot(data_df[feature], label=f'Distibution of {feature}', legend=True, )
    sns.lineplot(x=x, y=pdf, color='red', label=f'Normal distribution of {feature}')
    plt.ylabel('Density', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    plt.legend(edgecolor = 'r',
               facecolor = 'oldlace',
               ncol=1,
               fontsize=15,
               loc='center',
               bbox_to_anchor=(0.5, -0.5))

    plt.subplot(2, 2, 2)
    plt.title('Distribution for test', fontsize=13)
    mu, std = np.mean(closed_data[feature]), np.std(closed_data[feature])
    x = np.linspace(mu - 3*std, mu + 3*std, 100)
    pdf = norm.pdf(x, mu, std)
    sns.kdeplot(closed_data[feature], label=f'Distribution of {feature}', legend=True)
    sns.lineplot(x=x, y=pdf, color='red', label=f'Normal distribution of {feature}')
    plt.ylabel('Density', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    plt.legend(edgecolor = 'r',
               facecolor = 'oldlace',
               ncol=1,
               fontsize=15,
               loc='center',
               bbox_to_anchor=(0.5, -0.5))

    plt.subplot(2, 2, 3)
    stats.probplot(data_df[feature], plot=plt)
    plt.title('Probalitiplot for train', fontsize=13)
    plt.ylabel('Ordered values', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    
    plt.subplot(2, 2, 4)
    stats.probplot(closed_data[feature], plot=plt)
    plt.title('Probalitiplot for test', fontsize=13)
    plt.ylabel('Ordered values', fontsize=12)
    plt.xlabel(feature, fontsize=12);

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
plt.title('CORRELATION MATRIX (PEARSON).', fontsize=20)
sns.heatmap(data_df.corr(numeric_only=True), annot=True, cmap="coolwarm",  fmt='.2g', linewidth=4)
plt.grid(True);

In [ ]:
correlations_point_biserial = []
for column in data_df.columns:
    correlations_point_biserial.append(stats.pointbiserialr(data_df[column], target)[0])
df_with_point_biserial_corr = pd.DataFrame({'column': data_df.columns.tolist(),
                                            'correlation point biserial': correlations_point_biserial}) \
                                            .set_index('column')

In [ ]:
fig, ax = plt.subplots(figsize=(20,12))
plt.title('POINT-BISERIAL CORRELATION.', fontsize=20)
sns.heatmap(df_with_point_biserial_corr,
            annot=True,
            cmap="coolwarm",
            fmt='.2g',
            linewidth=4)
plt.ylabel('Features')
plt.xlabel('Values')
plt.grid(True)

In [ ]:
sns.scatterplot(data=data_df, x='0', y='2')

In [ ]:
scatter_matrix(data_df, alpha=0.2, figsize=(6, 6), diagonal="kde");

In [ ]:
f_for_model = ['0', '1', '2' ,'3']

In [ ]:
data_df = data_df[f_for_model]
closed_data = closed_data[f_for_model]

In [ ]:
scatter_matrix(data_df, alpha=0.2, figsize=(6, 6), diagonal="kde");

In [ ]:
data_df['target'] = target

In [ ]:
data_df['2'].describe()

In [ ]:
len(data_df[data_df['2'] < data_df['2'].quantile(q=0.25)]['2'])

In [ ]:
data_df = data_df[data_df['2'] >= data_df['2'].quantile(q=0.25)]

In [ ]:
data_df['2'].unique()

In [ ]:
target = data_df['target']

In [ ]:
scaler = RobustScaler()
data_df = scaler.fit_transform(data_df.drop(['target'], axis=1))
closed_data = scaler.transform(closed_data)

In [ ]:
grid = GridSearchCV(Ridge(), scoring='neg_mean_squared_error', cv=15, 
                    param_grid={'alpha': np.linspace(0,1, num=100),
                                'max_iter': [1000, 1500, 2000, 2500, 3000],
                                'random_state':[42]})

In [ ]:
grid.fit(data_df, target)

In [ ]:
grid.best_params_

In [ ]:
ridge = Ridge(random_state=42, alpha=0.18181818181818182, max_iter=1000)
ridge.fit(data_df, target)

In [ ]:
def get_predictions(model, eval_data, step=10):
    predicted_values = model.predict(eval_data)
    return predicted_values

Обращаем ваше внимание, предсказания округляются до сотых!

In [ ]:
predicted_values = np.round(get_predictions(model=ridge, eval_data=closed_data), 2)

assert predicted_values.shape == (closed_data.shape[0], ) # predictions should be just one-dimensional array

In [ ]:
# do not change the code in the block below
# __________start of block__________
def float_list_to_comma_separated_str(_list):
    _list = list(np.round(np.array(_list), 2))
    return ','.join([str(x) for x in _list])

submission_dict = {
    'predictions': float_list_to_comma_separated_str(predicted_values)
}
with open('submission_dict_final_p01.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
    
print('File saved to `submission_dict_final_p01.npy`')
# __________end of block__________

### Модель №2
Функция `my_transformation` принимает на вход матрицу объект-признак (`numpy.ndarray` типа `np.float`) и преобразует ее в новую матрицу. Данная функция может использовать только numpy-операции, а также арифметические действия.

Для примера доступна функция ниже. Она лишь добавляет новый признак, представляющий собой произведение первого и второго исходных признаков (считая с нуля).

In [2]:
def my_transformation(feature_matrix: np.ndarray):
    new = feature_matrix[:, :4]
    return new

In [3]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_target.npy

--2023-11-30 22:08:50--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 44928 (44K) [application/octet-stream]
Сохранение в: «hw_final_open_data.npy»

hw_final_open_data. 100%[===================>]  43,88K  --.-KB/s    за 0,04s   

2023-11-30 22:08:51 (1,16 MB/s) - «hw_final_open_data.npy» сохранён [44928/44928]

--2023-11-30 22:08:51--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubuserconte

In [4]:
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

In [5]:
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.3)

In [6]:
transformed_train_x = my_transformation(train_x)

In [7]:
lr = Ridge(random_state=42, alpha=0.18181818181818182, max_iter=1000)
lr.fit(transformed_train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(lr.predict(transformed_train_x), train_y):.5f}',
    f'validation mse = {mean_squared_error(lr.predict(my_transformation(valid_x)), valid_y):.5f}',
    sep='\n'
)

train mse =	 0.08574
validation mse = 0.09955


Обращаем ваше внимание, что параметры линейной модели будут округляться до __четырех знаков после запятой__. Это не должно сильно повлиять на качество предсказаний:

In [8]:
original_predictions = lr.predict(transformed_train_x)
rounded_predictions = transformed_train_x.dot(np.round(lr.coef_, 4)) + np.round(lr.intercept_, 4)


assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

Параметры вашей модели:

In [9]:
w_list = list(np.round(lr.coef_, 4))
print(f'w_submission = {list(np.round(lr.coef_, 4))}\nb_submission = {np.round(lr.intercept_, 4)}')

w_submission = [0.0418, 2.9833, 0.0, 2.4812]
b_submission = 2.965


Напоминаем, ваша модель не должна использовать более 15 параметров (14 весов плюс свободный член).

In [10]:
assert len(w_list) + 1 <= 15

##### Сдача второй части соревнования
Для сдачи вам достаточно отправить функцию `my_transformation` и параметры вашей модели в контест в задачу №2. Пример посылки доступен ниже. Имортирование `numpy` также необходимо.

In [11]:
# __________example_submission_start__________
import numpy as np
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0
    ] * feature_matrix[:, 1]
    return new_feature_matrix

w_submission = [-0.0027, -0.2637, 0.0, -0.1134, -0.0165, -0.9329, 0.0, 0.1293]
b_submission = 1.1312
# __________example_submission_end__________

На этом задание завершено. Поздравляем!